In [1]:
%run Classes_and_functions.ipynb

In [2]:
import time

In [3]:
ea=216
eb=137
f=1

In [4]:
p=2^ea*3^eb*f-1

In [5]:
F=GF(p^2,'z')

In [6]:
E=MontgomeryEC(6,1,F)
show(E)

<__main__.MontgomeryEC instance at 0x12ad9ff38>

In [7]:
'''
Generation of Pa,Qa and R (R is the secret someone has to prove)
'''
while true:
    while true:
        P=E.random_point()
        Pa=P*(3^eb*f)^2
        if Pa.order_torsion(2,ea)==2^ea:
            break
    show(Pa)
    while true:
        Q=E.random_point()
        Qa=Q*(3^eb*f)^2
        W=Pa.Weil_pairing(Qa,2^ea)
        if Qa.order_torsion(2,ea)==2^ea and check_order_F(W,2) == ea:
            break
    show(Qa)
    for i in range(10):
        RT=point_torsion(Pa,Qa,2,ea)
        R=RT[0]
        ta=RT[1]
        if (R*pow(2,ea-1)).x!=0:
            break
        if i==9:
            R.curve=0
    if R.curve!=0:
        break
show(R)

In [8]:
'''
Generation of the basis of the other torsion group Pb and Qb
'''
while true:
    P=E.random_point()
    Pb=P*(2^ea*f)^2
    if Pb.order_torsion(3,eb)==3^eb:
        break
show(Pb)
while true:
    Q=E.random_point()
    Qb=Q*(2^ea*f)^2
    W = Pb.Weil_pairing(Qb,3^eb)
    if Qb.order_torsion(3,eb)==3^eb and check_order_F(W,3) == eb:
        break
show(Qb)


In [9]:
'''
Generation of the secret isogeny
'''
phiA=iso_2_e(R,ea,E.a,E.b,Pb,Qb)
EaA=phiA[0][0]
EaB=phiA[0][1]
phiAPb=phiA[1][0]
phiAQb=phiA[1][1]

In [10]:
'''
Function for each iteration of the verification. This part is done by the verificator.
'''
@parallel(ncpus=20)
def verificacio(algo):
    Spt=point_torsion(Pb,Qb,3,eb)
    S=Spt[0]
    tb=Spt[1]
    phiB=iso_3_e(S,eb,E.a,E.b,Pa,Qa)
    EbA=phiB[0][0]
    EbB=phiB[0][1]
    phiBPa=phiB[1][0]
    phiBQa=phiB[1][1]
    Eb=MontgomeryEC(EbA,EbB,F)
    jEb=Eb.j_invariant()
    
    phiBR1=phiBQa*ta
    phiBR=phiBR1+phiBPa
    psiA=iso_2_e(phiBR,ea,EbA,EbB)
    EbaA=psiA[0][0]
    EbaB=psiA[0][1]
    Eba=MontgomeryEC(EbaA,EbaB,F)
    jEab=Eba.j_invariant()
    
    phiAS1 = phiAQb*tb
    phiAS = phiAS1+phiAPb
    b=randint(0,1)
    if (b==0):
        if(S.order_torsion(3,eb)!=pow(3,eb) or phiAS.order_torsion(3,eb) != pow(3,eb)):
            return False
        isog1=iso_3_e(S,eb,E.a,E.b)
        EbA1=isog1[0][0]
        EbB1=isog1[0][1]
        Eb1=MontgomeryEC(EbA1,EbB1,F)
        j1=Eb1.j_invariant()
        isog2=iso_3_e(phiAS,eb,EaA,EaB)
        EaA1=isog2[0][0]
        EaB1=isog2[0][1]
        Ea1=MontgomeryEC(EaA1,EaB1,F)
        j2=Ea1.j_invariant()
        if (j1!= jEb or j2!= jEab):
            return False
        return True
    else:
        if(phiBR.order_torsion(2,ea)!=pow(2,ea)):
            return False
        isog1=iso_2_e(phiBR,ea,EbA,EbB)
        EabA1=isog1[0][0]
        EabB1=isog1[0][1]
        Eab1 = MontgomeryEC(EabA1,EabB1,F)
        j1=Eab1.j_invariant()
        if (j1!= jEab):

            return False
        return(True)

In [11]:
'''
Verification
'''
start = time.time()
a=sorted(list(verificacio(range(20))))
for x in a:
    if x[1] != True:
        print('Authentication error')
        break
    if x==a[-1]:
        print('Correct authentication')
end=time.time()

Correct authentication


In [12]:
'''
Time elapsed for the verification
'''
print(end-start)

17.2886691093
